In [227]:
from collections import defaultdict


#initialize the variables needed and read in dataset
transactions = []
min_util = 1000000
utility_lists = {}
parsed_trans = []
TWU = defaultdict(int)
revised_trans = []
with open('Chicago_Crimes_2001_to_2017_utility.txt', 'r') as file:
    for line in file:
        transactions.append(line.strip())

FileNotFoundError: [Errno 2] No such file or directory: 'Chicago_Crimes_2001_to_2017_utility.txt'

In [211]:
#function to extract values from transaction and update item TWU values
def parse(line, tid):
    items, total_util, item_utils = line.split(':')
    #store items, total_util, item_utils
    items = list(map(int, items.strip().split()))
    total_util = float(total_util.strip())
    item_utils = list(map(float, item_utils.strip().split()))
    #add the transaction to new list of transactions with transaction id
    parsed_trans.append((tid, items, item_utils, total_util))
    #for each item in the transaction, update the TWU value in dictionary
    for item in items:
        TWU[item] += total_util


In [213]:
def revise():
    kept = {item for item in TWU if TWU[item] >= min_util} #keep an item only if its TWU higher or equal to minutil
    sorted_items = sorted(kept, key=lambda i: (TWU[i], i)) #sort them ascendingly by TWU
    ordering = {item: idx for idx, item in enumerate(sorted_items)} #establish ordering to revise transactions

    #iterate through parsed transactions to revise them
    for tid, items, utils, tu in parsed_trans:
        #only keep the items that pass the threshold and then sort them and store in new list of transactions
        revised = [(item, util) for item, util in zip(items, utils) if item in kept]
        revised.sort(key=lambda x: ordering[x[0]])
        if revised:
            revised_trans.append((tid, revised))

In [215]:
#utilitylistEntry class to store each entry in a UL
class UtilityListEntry:
    def __init__(self, tid, iu, ru):
        self.tid = tid
        self.iu = iu
        self.ru = ru

In [217]:
#build utility lists for all individual items
def build_item_UL():
    #iterate through the revised transactions
    for tid, revised in revised_trans:
        #extract items and their utils
        items = [item for item,_ in revised]
        utils = [util for _,util in revised]
        for i in range(len(items)):
            #find iutil and rutil
            item = items[i]
            iu = utils[i]
            ru = sum(utils[i+1:])
            #create utility list if needed
            if item not in utility_lists:
                utility_lists[item] = []
            #append the entry
            utility_lists[item].append(UtilityListEntry(tid, iu, ru))

In [219]:
#construct new set of ULs for deeper exploration
def construct(prefix, x, y):
    newUL = []
    mapY = {e.tid: e for e in y}
    mapP = {e.tid: e for e in prefix} if prefix else {}

    #iterate through the entries in the UL of x
    for Xentry in x:
        Yentry = mapY.get(Xentry.tid)
        #if there is a matched transaction
        if Yentry:
            #calculation if prefix exists
            if prefix:
                Pentry = mapP[Xentry.tid]
                iu = Xentry.iu + Yentry.iu - Pentry.iu
            #calculation if no prefix exists
            else:
                iu = Xentry.iu + Yentry.iu
            #rutil comes from the later item (y)
            ru = Yentry.ru
            #add the new entry
            newUL.append(UtilityListEntry(Xentry.tid, iu, ru))

    return newUL
    

In [221]:
def huiMiner(prefix, ULs):
    #iterate through the ULs passed in
    for i in range(len(ULs)):
        Xi, xUL = ULs[i]
        newPrefix = prefix + (Xi,)
        #find sumIU and sumTOT for evaluation of high utility
        sumIU = sum(e.iu for e in xUL)
        sumTOT = sum(e.ru + e.iu for e in xUL)

        #if sumIU over minutil, it is high utility, output
        if sumIU >= min_util:
            print(f"High Utility Itemset: {newPrefix}, Utility: {sumIU}")
        #if sumTOT over minutil, we should explore deeper
        if sumTOT >= min_util:
            extendedULs = []
            #moving through the rest of the ULs against "x", find ULs for the extensions using construct function
            for j in range(i+1, len(ULs)):
                Yj, yUL = ULs[j]
                newUL = construct(xUL, xUL, yUL)
                if newUL:
                    #add to list of ULs for deeper exploration
                    extendedULs.append((Yj, newUL))
            #recursive call to huiMiner() for further exploration down the tree
            huiMiner(newPrefix, extendedULs)
            
    

In [223]:
#runs the algorithm
def run():
    #parse the transactions
    for tid, line in enumerate(transactions):
        parse(line, tid)
    #revise them to run the recursion to find high utility itemsets
    revise()
    #find individual item ULs and sort them
    build_item_UL()
    ULs = sorted(utility_lists.items(), key=lambda x: TWU[x[0]])
    #start recursive call on empty tuple and initial set of ULs
    huiMiner(tuple(), ULs)

In [225]:
import time
import os
import matplotlib.pyplot as plt

#example run that records time

print("running HUIMiner at min util of: ")
print(min_util)


start = time.time()
run()
end = time.time()
elapsed = end - start

print("Time to run")
print(elapsed)
    

running HUIMiner at min util of: 
1000000
Time to run
0.00010800361633300781
